In [1]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from scipy.stats import rankdata
import time
from scipy.spatial.distance import pdist, squareform
import _pickle as cp

In [2]:
a = np.random.rand(10,10)

In [3]:
corr = np.corrcoef(np.transpose(a))
corr = 1 - corr
corr = np.around(a=corr, decimals=10)

In [6]:
num_r = num_c = 10
num = num_r
tril_id = np.tril_indices(num, k=-1)
rank = rankdata(corr[tril_id])
feature_ranking = np.zeros((num,num))
feature_ranking[tril_id] = rank
feature_ranking = feature_ranking + feature_ranking.T

In [7]:
for r in range(num_r):
    if r == 0:
        coordinate = np.transpose(np.vstack((np.zeros(num_c), range(num_c))))
    else:
        coordinate = np.vstack((coordinate, np.transpose(np.vstack((np.ones(num_c) * r, range(num_c))))))

In [8]:
coordinate

array([[0., 0.],
       [0., 1.],
       [0., 2.],
       [0., 3.],
       [0., 4.],
       [0., 5.],
       [0., 6.],
       [0., 7.],
       [0., 8.],
       [0., 9.],
       [1., 0.],
       [1., 1.],
       [1., 2.],
       [1., 3.],
       [1., 4.],
       [1., 5.],
       [1., 6.],
       [1., 7.],
       [1., 8.],
       [1., 9.],
       [2., 0.],
       [2., 1.],
       [2., 2.],
       [2., 3.],
       [2., 4.],
       [2., 5.],
       [2., 6.],
       [2., 7.],
       [2., 8.],
       [2., 9.],
       [3., 0.],
       [3., 1.],
       [3., 2.],
       [3., 3.],
       [3., 4.],
       [3., 5.],
       [3., 6.],
       [3., 7.],
       [3., 8.],
       [3., 9.],
       [4., 0.],
       [4., 1.],
       [4., 2.],
       [4., 3.],
       [4., 4.],
       [4., 5.],
       [4., 6.],
       [4., 7.],
       [4., 8.],
       [4., 9.],
       [5., 0.],
       [5., 1.],
       [5., 2.],
       [5., 3.],
       [5., 4.],
       [5., 5.],
       [5., 6.],
       [5., 7.],
       [5., 8.

In [17]:
num = num_r * num_c
cord_dist = np.zeros((num, num))    
for i in range(num_r):
    cord_dist[i, :] = np.sqrt(np.square(coordinate[i, 0] * np.ones(num) - coordinate[:, 0]) + \
         np.square(coordinate[i, 1] * np.ones(num) - coordinate[:, 1]))

In [18]:
tril_id = np.tril_indices(num, k=-1)
rank = rankdata(cord_dist[tril_id])
coor_ranking = np.zeros((num, num))
coor_ranking[tril_id] = rank
coor_ranking = coor_ranking + np.transpose(coor_ranking)
coordinate = np.int64(coordinate)

In [21]:
generate_coor_matrix = (coordinate[:, 0], coordinate[:, 1]), coor_ranking

In [22]:
max_step = 1000

In [23]:
source = feature_ranking.copy()
num = source.shape[0]
tril_id = np.array(range(num))
index = np.array(range(num))
index_record = np.empty((max_step + 1, num))
index_record.fill(np.nan)
index_record[0, :] = index.copy()

In [26]:
feature_ranking

array([[ 0., 29., 18., 40., 32., 45., 12., 23., 39., 21.],
       [29.,  0., 17., 13., 36.,  2., 20.,  3.,  9., 31.],
       [18., 17.,  0., 43., 41., 28.,  1., 26., 10., 37.],
       [40., 13., 43.,  0.,  8.,  5., 42.,  6., 15., 22.],
       [32., 36., 41.,  8.,  0., 27., 44., 25., 34., 19.],
       [45.,  2., 28.,  5., 27.,  0., 38.,  7.,  4., 14.],
       [12., 20.,  1., 42., 44., 38.,  0., 30., 11., 24.],
       [23.,  3., 26.,  6., 25.,  7., 30.,  0., 35., 16.],
       [39.,  9., 10., 15., 34.,  4., 11., 35.,  0., 33.],
       [21., 31., 37., 22., 19., 14., 24., 16., 33.,  0.]])

In [25]:
coor_ranking.shape

(100, 100)

JENSEN SHANNON DISTANCE METRIC 

In [47]:
from scipy.stats import entropy

In [48]:
def jensen_shannon_distance(x1: np.ndarray, x2: np.ndarray) -> float:
    m = 0.5 * (x1 + x2)
    kl_div_x1m = entropy(x1, m)
    kl_div_x2m = entropy(x2, m)
    return np.sqrt(0.5*kl_div_x1m + 0.5*kl_div_x2m)


In [49]:
def normalized_data(data):
    return data / data.sum(axis=1, keepdims=1)

In [50]:
data = np.array([[1,2,3],[4,5,6],[7,8,9]])
data

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [51]:
data = normalized_data(data.T)
data

array([[0.08333333, 0.33333333, 0.58333333],
       [0.13333333, 0.33333333, 0.53333333],
       [0.16666667, 0.33333333, 0.5       ]])

In [52]:
entropy(data[:,0], data[:,1],)

0.03740384776285771

In [55]:
res = np.zeros((3,3))
for i in range(3):
    for j in range(i):
        res[i,j] = jensen_shannon_distance(data[:,i], data[:, j])
        res[j,i] = res[i,j]
res

array([[0.        , 0.10693419, 0.14038336],
       [0.10693419, 0.        , 0.02301584],
       [0.14038336, 0.02301584, 0.        ]])